# Data Wrangling: Medical Supplier Dataset (main)

This notebook goes beyond basic cleaning and demonstrates professional-grade data wrangling techniques using the 63,000+ row medical supplier dataset. Each task includes output validation to show the impact of the transformation.

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load the Dataset

In [43]:
pd.set_option('display.max_columns', None)

df = pd.read_csv("../data/raw/Medical-Equipment-Suppliers.csv", low_memory=False)
print("Initial shape:", df.shape)
df.head()

Initial shape: (63112, 17)


,provider_id,acceptsassignement,participationbegindate,businessname,practicename,practiceaddress1,practiceaddress2,practicecity,practicestate,practicezip9code,telephonenumber,specialitieslist,providertypelist,supplieslist,latitude,longitude,is_contracted_for_cba
0,20561045,True,2007-11-01,HARTIG DRUG CO CORP,HARTIG DRUG STORE 5,11002 BARTELL BLVD,NaN,GALENA,IL,610368215,8157772700,Pharmacy,NaN,Enteral Nutrients|Walkers|Tracheotomy Supplies...,42.44484,-90.45593,False
1,20489003,False,2014-07-01,WAL-MART STORES EAST LP,WALMART PHARMACY 10-0674,1112 NASHVILLE PIKE,NaN,GALLATIN,TN,370667116,6154521110,Optician|Pharmacy,NaN,Enteral Nutrients|Oral Antiemetic Drugs|Blood ...,36.37498,-86.47526,False
2,20447711,False,2019-01-01,OHIO CVS STORES LLC,CVS PHARMACY #03306,1949 WEST MARKET STREET,NaN,AKRON,OH,443136910,3308675410,Pharmacy,NaN,Blood Glucose Monitors/Supplies (Mail Order)|P...,41.11353,-81.57820,False
3,20541431,False,2010-03-31,LUXOTTICA OF AMERICA INC,LENSCRAFERS #00855,2526 S THIRD ST,NaN,JACKSONVILLE,FL,322506024,9042472374,Optician,NaN,Prosthetic Lenses: Conventional Eyeglasses,30.26696,-81.38857,False
4,20469089,True,2016-12-15,SPECIALTY INFUSIONS INC,PRIME INFUSIONS,1624 BROADWAY,NaN,BROOKLYN,NY,112071026,7184434000,Pharmacy,NaN,External Infusion Pumps and/or Supplies|Osteog...,40.68483,-73.91404,False


## Drop Sparse Columns

In [44]:
threshold = len(df) * 0.8
sparse_cols = df.columns[df.isnull().sum() > threshold]
df.drop(columns=sparse_cols, inplace=True)
print("Dropped columns:", list(sparse_cols))
print("New shape:", df.shape)

Dropped columns: ['providertypelist']
New shape: (63112, 16)


## Convert Dates & Derive Account Age

In [45]:
df['participationbegindate'] = pd.to_datetime(df['participationbegindate'], errors='coerce')
df['account_age_years'] = 2025 - df['participationbegindate'].dt.year
print(df[['participationbegindate', 'account_age_years']].head())

  participationbegindate  account_age_years
0             2007-11-01                 18
1             2014-07-01                 11
2             2019-01-01                  6
3             2010-03-31                 15
4             2016-12-15                  9


## Group States into Regions

In [46]:
west = ['CA', 'WA', 'OR', 'NV']
south = ['TX', 'FL', 'GA']
df['region'] = df['practicestate'].apply(
    lambda x: 'West' if x in west else ('South' if x in south else 'Other')
)
print(df['region'].value_counts())

region
Other    44355
South    11163
West      7594
Name: count, dtype: int64


## Handle Specialities List (Split & Expand Preview)

In [47]:
df['specialitieslist'] = df['specialitieslist'].fillna('Unknown')
df['first_speciality'] = df['specialitieslist'].apply(lambda x: x.split('|')[0] if isinstance(x, str) else x)
print(df[['specialitieslist', 'first_speciality']].head())

    specialitieslist first_speciality
0           Pharmacy         Pharmacy
1  Optician|Pharmacy         Optician
2           Pharmacy         Pharmacy
3           Optician         Optician
4           Pharmacy         Pharmacy


## Simulate Price Column and Detect Outliers

In [48]:
np.random.seed(42)
df['price'] = np.random.uniform(5, 500, len(df))
outliers = df[df['price'] > 450]
print("Outliers detected (price > 450):", outliers.shape[0])

Outliers detected (price > 450): 6328


## Flag Incomplete or Low-Quality Records

In [49]:
df['data_quality_flag'] = df[['specialitieslist', 'supplieslist', 'latitude']].isnull().any(axis=1)
print("Rows flagged as incomplete:", df['data_quality_flag'].sum())

Rows flagged as incomplete: 49


## Aggregate: Count Providers Per Region

In [50]:
region_counts = df.groupby('region')['provider_id'].count()
print(region_counts)

region
Other    44355
South    11163
West      7594
Name: provider_id, dtype: int64


In [51]:
df

,provider_id,acceptsassignement,participationbegindate,businessname,practicename,practiceaddress1,practiceaddress2,practicecity,practicestate,practicezip9code,telephonenumber,specialitieslist,supplieslist,latitude,longitude,is_contracted_for_cba,account_age_years,region,first_speciality,price,data_quality_flag
0,20561045,True,2007-11-01,HARTIG DRUG CO CORP,HARTIG DRUG STORE 5,11002 BARTELL BLVD,NaN,GALENA,IL,610368215,8157772700,Pharmacy,Enteral Nutrients|Walkers|Tracheotomy Supplies...,42.44484,-90.45593,False,18,Other,Pharmacy,190.397359,False
1,20489003,False,2014-07-01,WAL-MART STORES EAST LP,WALMART PHARMACY 10-0674,1112 NASHVILLE PIKE,NaN,GALLATIN,TN,370667116,6154521110,Optician|Pharmacy,Enteral Nutrients|Oral Antiemetic Drugs|Blood ...,36.37498,-86.47526,False,11,Other,Optician,475.603582,False
2,20447711,False,2019-01-01,OHIO CVS STORES LLC,CVS PHARMACY #03306,1949 WEST MARKET STREET,NaN,AKRON,OH,443136910,3308675410,Pharmacy,Blood Glucose Monitors/Supplies (Mail Order)|P...,41.11353,-81.57820,False,6,Other,Pharmacy,367.337001,False
3,20541431,False,2010-03-31,LUXOTTICA OF AMERICA INC,LENSCRAFERS #00855,2526 S THIRD ST,NaN,JACKSONVILLE,FL,322506024,9042472374,Optician,Prosthetic Lenses: Conventional Eyeglasses,30.26696,-81.38857,False,15,South,Optician,301.335950,False
4,20469089,True,2016-12-15,SPECIALTY INFUSIONS INC,PRIME INFUSIONS,1624 BROADWAY,NaN,BROOKLYN,NY,112071026,7184434000,Pharmacy,External Infusion Pumps and/or Supplies|Osteog...,40.68483,-73.91404,False,9,Other,Pharmacy,82.229227,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63107,20678339,True,1984-10-01,VALLEY VISION CLINIC LTD,VALLEY VISION CLINIC,2200 S WASHINGTON ST,NaN,GRAND FORKS,ND,582016346,7017753135,Optometrist|Optician,Prosthetic Lenses: Conventional Eyeglasses|Pro...,47.89822,-97.04570,False,41,Other,Optometrist,412.018203,False
63108,20592113,False,2004-01-01,LINCARE INC,LINCARE,1442 W BUSINESS PARK DR,NaN,OREM,UT,840582223,8012243338,Medical Supply Company Other,Blood Glucose Monitors/Supplies (Mail Order)|P...,40.27440,-111.72970,False,21,Other,Medical Supply Company Other,344.064415,False
63109,20316683,True,2023-01-01,CVS PHARMACY INC,CVS PHARMACY #07458,3601 N BELT LINE RD,NaN,SUNNYVALE,TX,751829235,9722796997,Pharmacy,Heat & Cold Applications|Walkers|Oral Anticanc...,32.81465,-96.59612,False,2,South,Pharmacy,238.377114,False
63110,20474897,True,2016-12-07,PUBLIX SUPER MARKETS INC,PUBLIX PHARMACY #1775,2830 HIGHWAY 52,NaN,MONCKS CORNER,SC,29461,8438096083,Pharmacy,Oral Anticancer Drugs|Epoetin|Oral Antiemetic ...,33.16419,-80.01934,False,9,Other,Pharmacy,204.809964,False


## Save wrangled data for further analysis

In [52]:
df.to_csv('../data/processed/medical_suppliers_wrangled.csv', index=False)

## Summary of these tasks
- Sparse columns dropped: High missingness
- Dates parsed, account age derived
- Categorical grouping by region
- Lists split and simplified
- Outliers simulated and flagged
- Missing fields flagged for review
- Ready for downstream analysis or Excel export